### Spectrograms  
  
The spectrograms for this dataset have subsets for each row of the metadata like the EEGs do. Sub spectrograms are 300 x 401. They are 10 minute subsets of the overall spectrogram they belong to. The first column is time. The other 400 columns are split by region (RP, LP, RL, LL) and frequency in Hz.  
  
**Electrodes by Region:**  
  
- Right Parasagittal:  Fp2, F4, C4, P4, O2
- Left Parasagittal: Fp1, F3, C3, P3, O1 
- Right Lateral: Fp2, F8, T4, T6, O2 
- Left Lateral: Fp1, F7, T3, T5, O1  
  
The central electrodes are missing from the above groups. They're typically separated as their own set of channels. This can be seen in the section on the double banana montage [here](https://www.learningeeg.com/montages-and-technical-components).  
  
A tutorial on generating these spectrograms can be found [here](https://www.kaggle.com/code/cdeotte/how-to-make-spectrogram-from-eeg).  
  
Further discussion can be found [here](https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467877).  
  
The spectrograms provided with this dataset are spectrograms on these groups of electrodes rather than spectrograms for every individual electrode. In the tutorial, the proposed method for generating these spectrograms is to calculate the spectrogram for electrode chains in each group, and then average those resulting spectrograms.  
  
The electrode channels for LL are: (Fp1 ~ F7), (F7 ~ T3), (T3 ~ T5), (T5 ~ O2)  
  
LL spectrogram could be generated as follows:  
  
LL spec = (spec(Fp1 - F7) + spec(F7 - T3) + spec(T3 - T5) + spec(T5 - O2)) / 4  
  
For every electrode channel, you find the difference between the two signals, generate a spectrogram for that resulting signal, and then average the spectrograms.  
  
**Definitions:**  
  
- Montage: connection of electrodes; bipolar montages and referential montages  
- Bipolar Montage: electrodes' voltages are linked and compared to adjacent electrodes in a chain of electrodes  
- Double Banana: electrodes are linked to the electrode behind them  
- Parasagittal Chains: inside track from Fp to O (ignore midline electrodes)  
- Temporal Chains (Lateral): outside track from Fp to O  
- Central Chain: midline electrodes from Fz to Pz  
- Electrode Chain Signals: signal of second electrode subtracted from signal of first electrode  
- Positive Downward Deflection: first electrode more positive than second  
- Negative Upward Deflection: first electrode more negative than second  
- Phase Reversal: there will be an electrode of maximal voltage (the electrode which detects most of the voltage) and the surrounding signal tracings will either point away from or towards the electrode of maximal voltage (positive discharges cause surrounding tracings to point away from and negative discharges cause them to point towards)  
- End of Chain Phenomenon: Fp and O electrodes end the chain; Fp lacks an electrode in front to compare with and O lacks an electrode behind to compare with; the end of chain phenomenon is the fact you only see half of any possible phase reversals at these electrodes because they only have one possible pair and not two unlike the other electrodes in the chain
  
**How montages lead to EEG tracings:**  
  
The example is a -50mV discharge. Depending on its location and dipole, not every electrode will detect the full -50mV discharge. The electrode nearest to the discharge will detect the most, and as electrodes get further away, they'll detect less and less.  
  
**Next steps:**  
  
- Determine values for fs, nfft, nperseg, window length, etc.
- Write function to generate spectrograms from sub EEGs
- Visualize spectrograms  
- Denoising EEG signals first?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fastparquet, pyarrow

In [25]:
example = pd.read_parquet('train_spectrograms/{}.parquet'.format(df['spectrogram_id'][0]), engine = 'pyarrow')
sub_example = example[0:300]

In [26]:
sub_example

,time,LL_0.59,LL_0.78,LL_0.98,LL_1.17,LL_1.37,LL_1.56,LL_1.76,LL_1.95,LL_2.15,...,RP_18.16,RP_18.36,RP_18.55,RP_18.75,RP_18.95,RP_19.14,RP_19.34,RP_19.53,RP_19.73,RP_19.92
0,1,4.26,10.98,9.05,13.65,11.49,8.930000,18.840000,19.260000,19.240000,...,0.31,0.17,0.28,0.19,0.24,0.27,0.29,0.16,0.22,0.19
1,3,2.65,3.97,12.18,13.26,14.21,13.230000,9.650000,8.110000,11.280000,...,0.15,0.13,0.14,0.24,0.24,0.36,0.35,0.31,0.36,0.40
2,5,4.18,4.53,8.77,14.26,13.36,16.559999,19.219999,17.510000,22.650000,...,0.29,0.21,0.16,0.25,0.28,0.28,0.34,0.48,0.44,0.48
3,7,2.41,3.21,4.92,8.07,5.97,12.420000,10.820000,14.960000,21.809999,...,0.33,0.51,0.49,0.64,0.58,0.42,0.32,0.31,0.32,0.33
4,9,2.29,2.44,2.77,4.62,5.39,7.080000,9.840000,12.270000,14.410000,...,0.44,0.38,0.48,0.63,0.45,0.45,0.49,0.33,0.31,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,591,5.26,7.41,10.16,7.25,15.35,27.590000,28.650000,27.600000,30.690001,...,0.09,0.17,0.11,0.09,0.10,0.08,0.07,0.17,0.17,0.24
296,593,5.17,7.17,10.05,13.47,13.22,11.560000,29.790001,24.660000,23.379999,...,0.06,0.06,0.05,0.11,0.09,0.08,0.16,0.21,0.20,0.20
297,595,1.87,3.59,8.69,7.59,8.33,11.880000,12.690000,16.360001,24.240000,...,0.09,0.07,0.07,0.06,0.06,0.07,0.08,0.10,0.10,0.10
298,597,3.27,5.63,9.46,11.19,11.98,12.260000,9.240000,7.030000,8.660000,...,0.12,0.08,0.06,0.04,0.04,0.06,0.05,0.08,0.13,0.14
